# Import Padstack Definitions
This example shows how to import padstack definitions. This includes

- Download an example board
- Create a config file with hole information
- Create a config file with pad and anti-pad information

## Perform imports and define constants

Perform required imports.

In [1]:
import json
import toml
from pathlib import Path
import tempfile

from IPython.display import display
from ansys.aedt.core.examples.downloads import download_file
import pandas as pd

from pyedb import Edb

Define constants.

In [2]:
AEDT_VERSION = "2025.2"

Download the example PCB data.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys").name
file_edb = download_file(source="edb/ANSYS-HSD_V1.aedb", local_path=temp_folder)

## Load example layout.

In [4]:
edbapp = Edb(file_edb, edbversion=AEDT_VERSION)

C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\misc\decorators.py:55: UserWarning: Argument `edbversion` is deprecated for method `Edb`; use `version` instead.
  warnings.warn(
C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\generic\design_types.py:301: UserWarning: Your ANSYS AEDT version is eligible to gRPC version.You might consider switching to that version for better user experience.For more information please check this link: https://edb.docs.pyansys.com/version/dev/grpc_api/index.html
  warnings.warn(GRPC_GENERAL_WARNING, UserWarning)


PyEDB INFO: Star initializing Edb 10:29:04.390817


PyEDB INFO: Edb version 2025.2


PyEDB INFO: Logger is initialized. Log file is saved to C:\Users\ansys\AppData\Local\Temp\pyedb_ansys.log.


PyEDB INFO: legacy v0.67.1


...

PyEDB INFO: Database ANSYS-HSD_V1.aedb Opened in 2025.2


PyEDB INFO: Cell main Opened


PyEDB INFO: Builder was initialized.


PyEDB INFO: open_edb completed in 9.0563 seconds.


PyEDB INFO: EDB initialization completed in 9.1199 seconds.


## Create a config file with hole information

Keywords

- **name**. Name of the padstack definition.
- **hole_plating_thickness**. Hole plating thickness.
- **hole_range**. Supported types are 'through', 'begin_on_upper_pad', 'end_on_lower_pad', 'upper_pad_to_lower_pad'.
- **hole_parameters**.
  - **shape**. Supported types are 'circle', 'square', 'rectangle'.
  - Other parameters are shape dependent.
    - When shape is 'circle', supported parameter si 'diameter'.
    - When shape is 'square', supported parameter is 'size'.
    - When shape is 'rectangle', supported parameters are 'x_size', 'y_size'.

In [5]:
cfg = dict()
cfg["padstacks"] = {}
cfg["padstacks"]["definitions"] = [
    {
        "name": "v35h15",
        "hole_plating_thickness": "25um",
        "material": "copper",
        "hole_range": "through",
        "hole_parameters": {
            "shape": "circle",
            "diameter": "0.15mm",
        },
    }
]

In [6]:
df = pd.DataFrame(data=cfg["padstacks"]["definitions"])
display(df)

,name,hole_plating_thickness,material,hole_range,hole_parameters
0,v35h15,25um,copper,through,"{'shape': 'circle', 'diameter': '0.15mm'}"


In [7]:
cfg_file_path = Path(temp_folder) / "cfg.json"
with open(cfg_file_path, "w") as f:
    json.dump(cfg, f, indent=4, ensure_ascii=False)

Load config file

In [8]:
edbapp.configuration.load(cfg_file_path, apply_file=True)

PyEDB INFO: Updating nets finished. Time lapse 0:00:00


PyEDB INFO: Updating components finished. Time lapse 0:00:00


PyEDB INFO: Creating pin groups finished. Time lapse 0:00:00


PyEDB INFO: Placing sources finished. Time lapse 0:00:00


PyEDB INFO: Applying materials finished. Time lapse 0:00:00


...

PyEDB INFO: Applying modeler finished. Time lapse 0:00:00


PyEDB INFO: Placing ports finished. Time lapse 0:00:00


PyEDB INFO: Placing terminals completed in 0.0000 seconds.


PyEDB INFO: Placing probes finished. Time lapse 0:00:00


PyEDB INFO: Applying operations completed in 0.0000 seconds.


## Create a config file with pad information

Keywords

- **name**. Name of the padstack definition.
- **pad_parameters**.
  - **regular_pad**. List of pad definition per layer.
    - **layer_name**. Name of the layer.
    - **shape**. Supported types are 'circle', 'square', 'rectangle', 'oval', 'bullet'.
    - Other parameters are shape dependent.
      - When shape is 'circle', supported parameter si 'diameter'.
      - When shape is 'square', supported parameter is 'size'.
      - When shape is 'rectangle', supported parameters are 'x_size', 'y_size'.
      - When shape is 'oval', supported parameters are 'x_size', 'y_size', 'corner_radius'.
      - When shape is 'bullet', supported parameters are 'x_size', 'y_size', 'corner_radius'.

In [9]:
cfg = dict()
cfg["padstacks"] = {}
cfg["padstacks"]["definitions"] = [
    {
        "name": "v35h15",
        "pad_parameters": {
            "regular_pad": [
                {
                    "layer_name": "1_Top",
                    "shape": "circle",
                    "offset_x": "0.1mm",
                    "offset_y": "0.1mm",
                    "rotation": "0",
                    "diameter": "0.5mm",
                },
                {
                    "layer_name": "Inner1(GND1)",
                    "shape": "square",
                    "offset_x": "0.1mm",
                    "offset_y": "0.1mm",
                    "rotation": "45deg",
                    "size": "0.5mm",
                },
            ],
            "anti_pad": [{"layer_name": "1_Top", "shape": "circle", "diameter": "1mm"}],
        },
    }
]

In [10]:
cfg_file_path = Path(temp_folder) / "cfg.json"
with open(cfg_file_path, "w") as f:
    json.dump(cfg, f, indent=4, ensure_ascii=False)

Equivalent toml file looks like below 

In [11]:
toml_string = toml.dumps(cfg)
print(toml_string)

[padstacks]
[[padstacks.definitions]]
name = "v35h15"

[padstacks.definitions.pad_parameters]
[[padstacks.definitions.pad_parameters.regular_pad]]
layer_name = "1_Top"
shape = "circle"
offset_x = "0.1mm"
offset_y = "0.1mm"
rotation = "0"
diameter = "0.5mm"

[[padstacks.definitions.pad_parameters.regular_pad]]
layer_name = "Inner1(GND1)"
shape = "square"
offset_x = "0.1mm"
offset_y = "0.1mm"
rotation = "45deg"
size = "0.5mm"

[[padstacks.definitions.pad_parameters.anti_pad]]
layer_name = "1_Top"
shape = "circle"
diameter = "1mm"




Load config file

In [12]:
edbapp.configuration.load(cfg_file_path, apply_file=True)

PyEDB INFO: Updating nets finished. Time lapse 0:00:00


PyEDB INFO: Updating components finished. Time lapse 0:00:00


PyEDB INFO: Creating pin groups finished. Time lapse 0:00:00


PyEDB INFO: Placing sources finished. Time lapse 0:00:00


PyEDB INFO: Applying materials finished. Time lapse 0:00:00


...

PyEDB INFO: Applying modeler finished. Time lapse 0:00:00


PyEDB INFO: Placing ports finished. Time lapse 0:00:00


PyEDB INFO: Placing terminals completed in 0.0000 seconds.


PyEDB INFO: Placing probes finished. Time lapse 0:00:00


PyEDB INFO: Applying operations completed in 0.0000 seconds.


## Save and close Edb
The temporary folder will be deleted once the execution of this script is finished. Replace edbapp.save() with
edbapp.save_as("C:/example.aedb") to keep the example project.

In [13]:
edbapp.save()
edbapp.close()

PyEDB INFO: Save Edb file completed in 0.1110 seconds.


PyEDB INFO: Close Edb file completed in 0.0636 seconds.


True